In [1]:
# import libraries
import os

# load all environment variables
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

pipeline_root = os.environ['PIPELINE_ROOT']
base_image = os.environ.get("CONTAINER_IMAGE")
project_id = os.environ['PROJECT_ID']
region = os.environ['REGION']
service_account = os.environ['SERVICE_ACCOUNT']
artifact_repo = os.environ['ARTIFACT_REPO']
model_display_name = os.environ['MODEL_DISPLAY_NAME']
model_name = os.environ['MODEL_NAME']
endpoint_name = os.environ['ENDPOINT_NAME']
gcs_url = os.environ['GCS_URL']
train_ratio = float(os.environ['TRAIN_RATIO'])

In [2]:
# %%writefile ../components/load_data.py

from kfp.v2 import dsl
from kfp.v2.dsl import Dataset, Output
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

container_image = os.environ.get("CONTAINER_IMAGE", "python:3.8") # Fallback to a default image if not set

@dsl.component(base_image=base_image)
def load_data(
    gcs_url: str, 
    output_dataset: Output[Dataset]
):
    """Download data from a GCS URL and save it to the specified path as a Dataset."""
    
    # Logic-specific Imports
    from google.cloud import storage
    import pandas as pd

    # Extract bucket and blob info from GCS URL
    if not gcs_url.startswith("gs://"):
        raise ValueError("Invalid GCS URL format")
    parts = gcs_url[5:].split("/", 1)
    if len(parts) != 2:
        raise ValueError("Invalid GCS URL format")
    bucket_name, blob_name = parts

    # Create a GCS client
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Read the contents into Pandas DataFrame
    df = pd.read_csv(blob.open("rb"))

    # Save to the specified path as Dataset
    df.to_csv(output_dataset.path, index=False)
    output_dataset.metadata['dataset_metadata'] = {'format': 'csv'}

/var/folders/ry/26w9xk1j52d7mvm3ghqyhnq00000gn/T/ipykernel_60025/339227900.py:3: DeprecationWarning: The module `kfp.v2` is deprecated and will be removed in a futureversion. Please import directly from the `kfp` namespace, instead of `kfp.v2`.
  from kfp.v2 import dsl
